# Testing Notebook (Standalone)
This notebook tests the trained model on MNIST.

In [1]:

import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
    

In [2]:

# Re-define model for loading
class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(28*28, 10)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.linear(x)
    

In [3]:

# Load test data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
test_set = datasets.MNIST('data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=64)
    

In [4]:

# Evaluate function
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    return np.array(all_targets), np.array(all_preds)
    

In [5]:

# Plot confusion matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    cm_percent = np.nan_to_num(cm_percent)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_percent, annot=True, fmt=".1f", cmap="Blues",
                xticklabels=range(10), yticklabels=range(10))
    plt.title("Confusion Matrix (%)")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()
    plt.show()
    

In [13]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LogisticRegression()
model.load_state_dict(torch.load("logistic.pth", map_location=device))
model.to(device)
y_true, y_pred = evaluate_model(model, test_loader, device)
plot_confusion_matrix(y_true, y_pred)
print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    

AttributeError: 'numpy.ndarray' object has no attribute 'results'

In [11]:
def plot_confusion_matrix(self, model_name):
    y_true = self.results[model_name]["true_labels"]
    y_pred = self.results[model_name]["pred_labels"]
    labels = list(range(10))  # For MNIST: 0 to 9

    cm = confusion_matrix(y_true, y_pred, labels=labels)

    # Normalize rows (true labels) to percentages
    cm_percent = cm.astype("float") / cm.sum(axis=1, keepdims=True) * 100
    cm_percent = np.nan_to_num(cm_percent)  # avoid NaNs for any empty rows

    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(cm_percent, interpolation="nearest", cmap=plt.cm.Blues)

    # Colorbar
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel("Percentage (%)", rotation=-90, va="bottom")

        # Set labels
    ax.set(
    xticks=np.arange(len(labels)),
    yticks=np.arange(len(labels)),
    xticklabels=labels,
    yticklabels=labels,
    xlabel="Predicted label",
    ylabel="True label",
    title=f"Confusion Matrix (%) - {model_name}"
    )

    # Rotate tick labels
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Annotate each cell with percentage
    for i in range(len(labels)):
        for j in range(len(labels)):
            percentage = cm_percent[i, j]
            ax.text(
                j, i, f"{percentage:.1f}",
                ha="center", va="center",
                color="white" if percentage > 50 else "black"
            )

    plt.tight_layout()
    plt.show()